# ห้ามลบ

- ในหน้านี้พลเป็นคนทำ
- ควรแก้ ในการยิง API ผมได้ใส่ parametor link ให้คุณเอา link สับย่อยมาใส่

##สิ่งที่แก้ไข

1.ทำการลดการ import

2.ลบหรือยุบตัวแปรที่ไม่ได้ใช้ออกเพื่อให้เข้าใจง่าย

3.เปลี่ยนแปลง API  ให้สามารถส่งลิ้งก์มาด้วย

4.ได้ทำการเปลี่ยน nan เป็น "" หรือ 0 สำเร็จแล้ว

5.uri_API  เพิ่มเส้น APi ไว้ด้านบนใต้ import เพื่อให้ง่ายในการแก้ไข

In [6]:
from tkinter import *
import tkinter as tk
from tkinter import messagebox,scrolledtext,ttk
import webbrowser,threading,subprocess,sys
# ++++++++++++++++++++++++++++++ gui ++++++++++++++++++++

# ++++++++++++++++++++++++++++++ main ++++++++++++++++++++ import 
import requests,os,json,time,pandas as pd,pyautogui,shutil,pyperclip,keyboard as ky , datetime as dt
from datetime import datetime
from fnmatch import fnmatch
from pynput import keyboard,mouse
from data_address import address as ad
uri_API = 'https://e5da-14-207-201-178.ngrok-free.app/';
# path
bot_shopee = r'\Bot_shopee';
path_file = os.getcwd();
drag_data = os.path.abspath(os.path.join(path_file, os.pardir))
data_shopee = r'\Data_shopee';
data_shopee_xlsx = r'\shopee.xlsx';
un_process = r'\Unprocess';
data_link = r'\Data_link\data_link_all.json';
desk_top = 8;



In [3]:
import requests,pandas as pd
uri_API = ''
data_link_for_lazada  = {
    0: 'อุปกรณ์-อิเล็กทรอนิกส์',
    1: 'อุปกรณ์เสริม-อิเล็กทรอนิกส์', 
    2: 'ทีวีและเครื่องใช้ในบ้าน', 
    3: 'สุขภาพและความงาม', 
    4: 'ทารกและของเล่น', 
    5: 'ของชำและสัตว์เลี้ยง', 
    6: 'บ้านและไลฟ์สไตล์', 
    7: 'แฟชั่นและเครื่องประดับผู้หญิง', 
    8: 'แฟชั่นและเครื่องประดับผู้ชาย',
    9: 'กีฬาและการเดินทาง',
    10: 'ยานยนต์และรถจักรยานยนต์'
    }
# head_excel
header_2 = ['col-xs-2-4 href', 'Fd4QmV src', 'FTxtVW',
       'customized-overlay-image src', 'DgXDzJ', 'bPcAVl', 'k9JZlv',
       'bx++ig 2', 'k9JZlv 2', 'OwmBnn', 'JVW3E2', 'hxLzax']
header = ['_95X4G href', 'jBwCF src', 'jBwCF src 2','RfADt', 'ooOxS','_1cEkb', 'qzqFw', 'oa6ri']
header_Values = {
    'col-xs-2-4 href':"product",
    'Fd4QmV src':"image_product_1",
    'FTxtVW':"discount",
    'customized-overlay-image src':"image_product_2",
    'DgXDzJ':"data_product", 
    'bPcAVl':"price_before",
    'k9JZlv':"price_product_1",
    'bx++ig 2':"Emoji",
    'k9JZlv 2':"price_product_2",
    'OwmBnn':"sold",
    'JVW3E2':"place", 
    'hxLzax':"Recommended_shops"
}
status_run_program = False

In [8]:
def postAPI_DB(data,id_shop,title_group,i1,link):
    """
    data: text ที่ทำการ += ในตัวแปร success_data_text
    id_shop : shop1_1_1
    title_group:หมวดหมู่กลุ่ม
    i1:กลุ่มหลัก 1 
    link: link หมวดหลัก
    """
    try:
        if(status_run_program):# หยุดทำงาน
            return
        response = requests.post(
            f"{uri_API}addb?id={id_shop}&&web=shopee&&group={title_group}&&title_group={title_group}&&link={link}",
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return response.text
    except:
        return {"status":404,"message":"POST API ERROR."}


In [7]:
def Check_header(path_file):
    try:
        check_data=[]
        df = pd.read_excel(path_file)
        print(df.columns[0])
        # is_subset = all(item in df.columns for item in header);
        # print("Check_data : พบข้อมูลทั้งหมดใน Excel ",is_subset);
        for i in range(len(header_2)):
            for excel in range(len(df.columns)):
                if(status_run_program):# หยุดทำงาน
                    return
                if(header_2[i]==df.columns[excel]):
                    check_data.append(df.columns[excel]);
        print("Check_data : ข้อมูลที่มีทั้งหมด ");
        return check_data; 
    except Exception as e:
        print("Check_data : ข้อมูล Excel ",e); 

In [10]:
def is_thai(text):
    thai_unicode_range = (0x0E00, 0x0E7F)
    return all(ord(char) in range(thai_unicode_range[0], thai_unicode_range[1] + 1) for char in text)
def postAPI_DB(data,id_shop,link,date,time,website):
    """
    data: text ที่ทำการ += ในตัวแปร success_data_text
    id_shop : shop1_1_1
    title_group:หมวดหมู่กลุ่ม
    i1:กลุ่มหลัก 1 
    link: link หมวดหลัก
    """
    try:
        response = requests.post(
            f'{uri_API}addb?idshop={id_shop}&&link={link}&&date={date}&&time={time}&&website={website}',
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return response
    except:
        return {"status":404,"message":"POST API ERROR."}
def convert_to_integer(s):
    if 'พัน' in s:
        return int(float(s.replace('พัน', '')) * 1000);
    elif 'ล้าน' in s:
        return int(float(s.replace('ล้าน', '')) * 1000000);
    else:
        return int(s)
    

def get_datetime():
  now = datetime.now()
  date_str = now.strftime("%Y-%m-%d")
  time_str = now.strftime("%H:%M:%S")
  return {
    "date": date_str,
    "time": time_str,
}
def data_process(path_file,i1,i2,i3,group,link):
    # try:
        find = pd.read_excel(path_file);
        header = Check_header(path_file)
        data_all=[];
        df = pd.read_excel(path_file)
        num_rows, num_columns = df.shape
        Data_everthing=[];
        success_data_text = ""
        for i in range(num_rows):
            dt = get_datetime()
            Date = dt['date']
            Time = dt['time']
            data_process = {
                "product":[],
                "price_product_2":[],
                "price_product_1":[],
                "image_product_1":[],
                "discount":[],
                "image_product_2":[],
                "data_product":[],
                "price_before":[],
                "Emoji":[],
                "sold":[],
                "place":[],
                "Recommended_shops":[],
                "count_review":[],
                "maket":[],
                'gruop':[],
                'date':[],
                'time':[],
                'link':[]
            }
            data = "Product_"+str(i+1);
            Product = {
                data:{
                }
            }
            # เข้าถึงข้อมูลแต่ละชิ้น
            for j in range(len(header)):
                data_input = str(find[header[j]][i]);
                # print(data_input)
                data_process[header_Values[header[j]]]=data_input;
            Product[data]=data_process;
            Product[data]["maket"] = "shopee";
            Product[data]["group"] = group;
            # ****************************************************************
            Product[data]['date'] = Date
            Product[data]['time'] = Time
            Product[data]['link'] = link
            # ****************************************************************
            price_product1 = float(Product[data]["price_product_1"].replace(",",""));
            price_product2 = float(Product[data]["price_product_2"].replace(",",""));
            discount = (Product[data]["discount"]=='nan')and "" or Product[data]["discount"];
            price_product1 = (price_product1<=0)and "0" or price_product1;
            price_product2 = (price_product2<=0)and "0" or price_product2;
            if is_thai(Product[data]["place"]):
                address = (Product[data]["place"]=='nan')and "" or Product[data]["place"];
                address=address.replace("จังหวัด","");
            else:
                address = (Product[data]["place"]=='nan')and " " or ad[Product[data]["place"]]
            if(len(Product[data]["sold"])>0):
                sold = (Product[data]["sold"]=='nan')and "" or Product[data]["sold"];
            else:
                sold = "0"
            if("ขายแล้ว" in sold):
                sold = convert_to_integer(sold.split(" ")[1])
            price_before = (Product[data]["price_before"]=='nan')and "" or Product[data]["price_before"]
            # ***************************ไอดีสินค้าหลัก*************************************
            id_shop = "shop"+str(i1)+"_"+str(i2)+"_"+str(i3);
            # ****************************************************************
            data_all.append(Product);

            # ถ้าข้อมูลครบ 60 ค่อยบันทึก .json และส่ง API
            success_data_text += f'APRODUCT:::maket:::{Product[data]["maket"]},'
            success_data_text += f'group:::{group},'
            success_data_text += f'product:::{Product[data]["product"]},'
            success_data_text += f'price_product_2:::{price_product2},' 
            success_data_text += f'price_product_1:::{price_product1},'
            success_data_text += f'image_product_1:::{Product[data]["image_product_1"]},'
            success_data_text += f'discount:::{discount},'
            success_data_text += f'image_product_2:::{Product[data]["image_product_2"]},'
            success_data_text += f'data_product:::{Product[data]["data_product"]},'
            success_data_text += f'price_before:::{price_before},'
            success_data_text += f'Emoji:::{Product[data]["Emoji"]},'
            success_data_text += f'sold:::{sold},'
            success_data_text += f'place:::{address},'
            success_data_text += f'Recommended_shops:::{Product[data]["Recommended_shops"]},'
            success_data_text += f'date:::{Product[data]["date"]},'
            success_data_text += f'time:::{Product[data]["time"]},'
            success_data_text += f'link:::{Product[data]["link"]}'
            if(i==num_rows-1):
                print(success_data_text);
                print(postAPI_DB(success_data_text,id_shop,link,Date,Time,'Shopee'));
        print("data_process : True")
    # except Exception as e:
    #     print("data_process : False",e)

data_process(path_file+data_shopee+r'/data_1_1_1.xlsx',1,1,1,"test","test");

col-xs-2-4 href
Check_data : ข้อมูลที่มีทั้งหมด 
APRODUCT:::maket:::shopee,group:::test,product:::https://shopee.co.th/%E0%B8%9E%E0%B8%A3%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%AA%E0%B9%88%E0%B8%87-Nintendo-Switch%E2%84%A2-Super-Mario-Bros.-Wonder-(By-ClaSsIC-GaME)-i.106689123.22071931268?sp_atk=9c410f0e-eff7-4e9c-8889-8ee2c295e319&xptdk=9c410f0e-eff7-4e9c-8889-8ee2c295e319,price_product_2:::2749.0,price_product_1:::1649.0,image_product_1:::https://down-th.img.susercontent.com/file/th-11134207-7r98p-ln40ujg8nxvt9e_tn,discount:::-29%,image_product_2:::https://down-th.img.susercontent.com/file/th-50009109-300ddf00d967483819649fe53299280c,data_product:::พร้อมส่ง | Nintendo Switch™ Super Mario Bros. Wonder (By ClaSsIC GaME),price_before:::-,Emoji:::฿,sold:::349,place:::กรุงเทพมหานคร,Recommended_shops:::nan,date:::2024-02-10,time:::22:19:18,link:::testAPRODUCT:::maket:::shopee,group:::test,product:::https://shopee.co.th/PlayStation%E2%84%A2-PS5-The-Last-of-Us-Part-II-Remastered-(By-ClaSsIC-GaME)-

In [11]:


import requests,pandas as pd
from data_address import address as ad
uri_API = 'https://b18f-223-206-131-122.ngrok-free.app/';
header = ['col-xs-2-4 href', 'Fd4QmV src', 'FTxtVW',
       'customized-overlay-image src', 'DgXDzJ', 'bPcAVl', 'k9JZlv',
       'bx++ig 2', 'k9JZlv 2', 'OwmBnn', 'JVW3E2', 'hxLzax']
header_Values = {
    'col-xs-2-4 href':"product",
    'Fd4QmV src':"image_product_1",
    'FTxtVW':"discount",
    'customized-overlay-image src':"image_product_2",
    'DgXDzJ':"data_product", 
    'bPcAVl':"price_before",
    'k9JZlv':"price_product_1",
    'bx++ig 2':"Emoji",
    'k9JZlv 2':"price_product_2",
    'OwmBnn':"sold",
    'JVW3E2':"place", 
    'hxLzax':"Recommended_shops"
}
def check_data(path_file):
    try:
        header = ['col-xs-2-4 href', 'Fd4QmV src',
       'customized-overlay-image src', 'DgXDzJ', 'bPcAVl', 'k9JZlv',
       ]
       
        df = pd.read_excel(path_file)
        is_subset = all(item in df.columns for item in header);
        print("Check_data : ",is_subset);
        return is_subset; 

    except Exception as e:
        
        print("Check_data : ",e);  
# Read Excell
def postAPI_DB(data,id_shop,title_group,i1,link):
    """
    data: text ที่ทำการ += ในตัวแปร success_data_text
    id_shop : shop1_1_1
    title_group:หมวดหมู่กลุ่ม
    i1:กลุ่มหลัก 1 
    link: link หมวดหลัก
    """
    try:
        response = requests.post(
            f"{uri_API}addb?id={id_shop}&&web=shopee&&group={title_group}&&title_group={title_group}&&link={link}",
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return {"status":200,"message":"POST API SUCCESS."}
    except:
        return {"status":404,"message":"POST API ERROR."}
def data_process(path_file,i1,i2,i3,group,link):
    try:
        find = pd.read_excel(path_file);
        data_all=[];
        df = pd.read_excel(path_file)
        num_rows, num_columns = df.shape
        Data_everthing=[];
        success_data_text = ""
        for i in range(num_rows):
            data_process = {
                "product":[],
                "price_product_2":[],
                "price_product_1":[],
                "image_product_1":[],
                "discount":[],
                "image_product_2":[],
                "data_product":[],
                "price_before":[],
                "Emoji":[],
                "sold":[],
                "place":[],
                "Recommended_shops":[],
                "count_review":[],
                "maket":[],
                "group":[]
            }
            data = "Product_"+str(i+1);
            Product = {
                data:{
                }
            }
            # เข้าถึงข้อมูลแต่ละชิ้น
            for j in range(12):
                data_input = str(find[header[j]][i]);
                # print(data_input)
                data_process[header_Values[header[j]]]=data_input;
            Product[data]=data_process;
            Product[data]["maket"] = "shopee";
            Product[data]["group"] = group;
            price_product1 = float(Product[data]["price_product_1"].replace(",",""));
            price_product2 = float(Product[data]["price_product_2"].replace(",",""));
            discount = (Product[data]["discount"]=='nan')and "" or Product[data]["discount"];
            price_product1 = (price_product1<=0)and "0" or price_product1;
            price_product2 = (price_product2<=0)and "0" or price_product2;
            address = (Product[data]["place"]=='nan')and "" or Product[data]["place"];
            sold = (Product[data]["sold"]=='nan')and "" or Product[data]["sold"];
            price_before = (Product[data]["price_before"]=='nan')and "" or Product[data]["price_before"]
            # ***************************ไอดีสินค้าหลัก*************************************
            id_shop = "shop"+str(i1)+"_"+str(i2)+"_"+str(i3);
            # ****************************************************************
            success_data_text += f'APRODUCT:::maket:::{Product[data]["maket"]}, group:::{group}, product:::{Product[data]["product"]}, price_product_2:::{price_product2}, price_product_1:::{price_product1}, image_product_1:::{Product[data]["image_product_1"]}, discount:::{discount}, image_product_2:::{Product[data]["image_product_2"]}, data_product:::{Product[data]["data_product"]}, price_before:::{price_before}, Emoji:::{Product[data]["Emoji"]}, sold:::{sold}, place:::{address}, Recommended_shops:::{Product[data]["Recommended_shops"]}'
            data_all.append(Product);
            success_data = {
                 "id":id_shop,
                 "data":data_all
            }
            # ถ้าข้อมูลครบ 60 ค่อยบันทึก .json และส่ง API
            if(i==num_rows-1):
                print(success_data_text); #ข้อมูลที่จะส่งไป API
                print(postAPI_DB(success_data_text,id_shop,group,i1,link));
        print("data_process : True")
    except Exception as e:
        print("data_process : False",e)
data_process('../Bot_shopee/Data_shopee/data_1_1_1.xlsx',1,1,1,'อุปกรณ์-อิเล็กทรอนิกส์',"")

APRODUCT:::maket:::shopee, group:::อุปกรณ์-อิเล็กทรอนิกส์, product:::https://shopee.co.th/%F0%9F%94%A5HOT%E0%B9%84%E0%B8%81%E0%B9%88%E0%B8%95%E0%B8%B1%E0%B8%99-%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%B1%E0%B8%99%E0%B8%95%E0%B8%B5-(6%E0%B8%AB%E0%B8%A1%E0%B8%B1%E0%B8%94-GOD-%E0%B8%AA%E0%B8%B8%E0%B9%88%E0%B8%A1%E0%B8%9C%E0%B8%A5%E0%B8%95%E0%B8%B7%E0%B9%88%E0%B8%99%E0%B9%80%E0%B8%97%E0%B8%9E%E0%B9%86%E0%B8%9E%E0%B8%A3%E0%B9%89%E0%B8%AD%E0%B8%A1PVP)-%E0%B8%AA%E0%B8%B8%E0%B9%88%E0%B8%A1%E0%B8%AD%E0%B8%B7%E0%B9%88%E0%B8%99%E0%B8%AD%E0%B8%B5%E0%B8%81%E0%B8%A1%E0%B8%B2%E0%B8%81%E0%B8%A1%E0%B8%B2%E0%B8%A2-%E0%B9%84%E0%B8%81%E0%B9%88%E0%B8%95%E0%B8%B1%E0%B8%99-%E0%B9%82%E0%B8%A3%E0%B8%9A%E0%B8%AD%E0%B8%81-%E0%B8%9A%E0%B8%A5%E0%B9%87%E0%B8%AD%E0%B8%81%E0%B8%9F%E0%B8%B8%E0%B8%95-blox-fruit-roblox-i.1020500864.23171691914?sp_atk=d402a59d-2a93-4eff-a9f5-9ac09e455053&xptdk=d402a59d-2a93-4eff-a9f5-9ac09e455053, price_product_2:::nan, price_product_1:::49.0, image_product_1:::https://down-th.img.susercontent.